<h1><center>Hito 7</center></h1>

<h1><center>Impacto del Bit-Rate</center></h1>

&nbsp;

<center>
Álvaro Garcı́a Garcı́a<br>
Álvaro José Martı́nez Sánchez<br>
José Francisco Castillo Berenguel<br>
</center>
    
    
<hr style="border:1px solid gray"> </hr>


# Introducción


En el hito 6 se estudiaron los efectos de la latencia y su variación (jitter). No obstante, existen otros factores que también afectan a la conexión. En este hito se realizará el estudio de la tasa de bits (Bit rate). 

Este valor de bit-rate depende del ancho de banda y de la congestión de la red. La congestión de la red resulta ser un factor arbitrario en la práctica, pero el ancho de banda puede considerarse constante.

La idea consistirá en obtener datos sobre el efecto del bit rate para posteriormente realizar la emulación con e comando 
`tc`

# Estimación del bit-rate en un enlace de internet


## Fundamentos

## Consideraciones

Se pide realizar distintos tipos de ping donde se incluye realizarlos con el mayor y menor payload. Para que los ping se realicen satisfactoriamente es necesario tener en cuenta estos efectos

1. La cabecera de ICMPv4 ocupa 8 B.

2. El valor mínimo de payload es 0. Sin embargo no se contabilizan los tiampos para payload menores de 16B (24B con la cabecera).

3. El valor máximo de payload es 65507.


## Procedimiento

Vamos a realizar el mismo procedimiento para 3 conexiones distintas:

1. Localhost
2. Red local
3. Conexión internet

Para ello realizamos ping con el menor y mayor payload posible 16B y 65507B respectivamente. Una vez hecho, cargamos los resultados. Sin embargo al final del ping se ofrecen las estadísticas del mismo y por tanto no hay que realizar cálculos. Obtenemos los datos:

In [34]:
!export LC_NUMERIC=en_US.UTF-8

!tail -n 1 ping-minimo-internet.dat | cut -f 2 -d "=" | cut -f 2 -d "/" > average-internet.dat
!tail -n 1 ping-maximo-internet.dat | cut -f 2 -d "=" | cut -f 2 -d "/" >> average-internet.dat

!tail -n 1 ping-minimo-localhost.dat | cut -f 2 -d "=" | cut -f 2 -d "/" > average-localhost.dat
!tail -n 1 ping-maximo-localhost.dat | cut -f 2 -d "=" | cut -f 2 -d "/" >> average-localhost.dat

!tail -n 1 ping-minimo-redlocal.dat | cut -f 2 -d "=" | cut -f 2 -d "/" > average-redlocal.dat
!tail -n 1 ping-maximo-redlocal.dat | cut -f 2 -d "=" | cut -f 2 -d "/" >> average-redlocal.dat

Cargamos los datos en python

In [37]:
import numpy as np   
    
rtt_localhost = np.loadtxt("average-localhost.dat")
rtt_redlocal = np.loadtxt("average-redlocal.dat")
rtt_internet = np.loadtxt("average-internet.dat")

print(rtt_localhost)
print(rtt_redlocal)
print(rtt_internet)
print()
tp_localhost = rtt_localhost[:]/2
tp_redlocal = rtt_redlocal[:]/2
tp_internet = rtt_internet[:]/2

print(tp_localhost)
print(tp_redlocal)
print(tp_internet)
print()
tt_localhost = (rtt_localhost[:] - 2.0 * tp_localhost[:]) / 2.0
tt_internet = (rtt_internet[0] - 2.0 * tp_internet[0]) / 2.0

print(tt_localhost)
print(tt_internet)
print(rtt_internet[0])
print(tp_internet[0])
print((rtt_internet[0] - 2.0 * tp_internet[0]))


[0.037 0.163]
[18.557 45.203]
[27.731 56.002]

[0.0185 0.0815]
[ 9.2785 22.6015]
[13.8655 28.001 ]

[0. 0.]
0.0
27.731
13.8655
0.0


TypeError: empty() missing required argument 'shape' (pos 1)